In [ ]:
pip install "pycozo[embedded,pandas]"

In [ ]:
from pycozo.client import Client
import pandas as pd
from pycozo.client import Client
client = Client('rocksdb', 'datafile.db')

In [ ]:
#create abstraction levels "replyedComment"

import pandas as pd

# 1) Charger les données depuis CozoDB
node_label_df = pd.DataFrame(client.run('?[id_n,ln] := *node_label[id_n,ln]'))
edge_label_df = pd.DataFrame(client.run('?[id_e,ln] := *edge_label[id_e,ln]'))
edge_df       = pd.DataFrame(client.run('?[id_e,ns,nd] := *edge[id_e,ns,nd]'))

# 2) Filtrer les commentaires et les arêtes replyOf
comment_ids = set(node_label_df[node_label_df['ln'] == 'comment']['id_n'])
reply_edges = edge_label_df[edge_label_df['ln'] == 'replyOf']['id_e']
reply_df    = edge_df[edge_df['id_e'].isin(reply_edges)]

# 3) Construire les  replyedComment 
chain = []
used_edges = set()
for _, row in reply_df.iterrows():
    if row['ns'] in comment_ids and row['nd'] in comment_ids:
        chain.append(row)
        used_edges.add(row['id_e'])
        break
while len(chain) < 89:
    prev_nd = chain[-1]['nd']
    next_rows = reply_df[
        (reply_df['ns'] == prev_nd) &
        (~reply_df['id_e'].isin(used_edges))
    ]
    if next_rows.empty:
        break
    row = next_rows.iloc[0]
    chain.append(row)
    used_edges.add(row['id_e'])

# 4) Générer les IDs réifiés et leurs composants
start_rn = 3926862
rn = start_rn
prev_rn = None

reified_nodes = []  # [[id_rn], ...]
n_comp_node   = []  # [[id_rn, id_n], ...]
n_comp_edge   = []  # [[id_rn, id_e], ...]

for i, row in enumerate(chain):
    reified_nodes.append([rn])
    if i == 0:
        # Niveau 0 : deux commentaires
        n_comp_node.append([rn, row['ns']])
        n_comp_node.append([rn, row['nd']])
    else:
        # Niveaux suivants : réifié précédent + nouveau commentaire
        n_comp_node.append([rn, prev_rn])
        n_comp_node.append([rn, row['nd']])
    # Attacher l’arête replyOf
    n_comp_edge.append([rn, row['id_e']])
    prev_rn = rn
    rn += 1

# 5) Fonctions pour formater les commandes Cozo
def format_cozo(rel, rows, cols):
    entries = ",".join(
        "[" + ",".join(str(x) for x in row) + "]"
        for row in rows
    )
    return f"?[{','.join(cols)}] <- [{entries}] :put {rel} {{{','.join(cols)}}}"

# 6) Imprimer les scripts Cozo
# a) déclarer les reified_node
print(format_cozo("reified_node",       reified_nodes, ["id_rn"]))
# b) rattacher composants nœuds
print(format_cozo("n_composed_of_node", n_comp_node,   ["id_rn","id_n"]))
# c) rattacher composants arêtes
print(format_cozo("n_composed_of_edge", n_comp_edge,   ["id_rn","id_e"]))
# d) ajouter le label replyedComment à chacun
#    on profite de la même liste reified_nodes
labels = [[rn_id, "replyedComment"] for (rn_id,) in reified_nodes]
print(format_cozo("node_label",         labels,        ["id_n","ln"]))


In [ ]:
res = client.run(format_cozo("reified_node",       reified_nodes, ["id_rn"]))
res

In [ ]:
res = client.run(format_cozo("n_composed_of_node", n_comp_node,   ["id_rn","id_n"]))
res

In [ ]:
res = client.run(format_cozo("n_composed_of_edge", n_comp_edge,   ["id_rn","id_e"]))
res

In [ ]:
res = client.run('?[id_n,ln] <- [[3926862,"replyedComment"],[3926863,"replyedComment"],[3926864,"replyedComment"],[3926865,"replyedComment"],[3926866,"replyedComment"],[3926867,"replyedComment"],[3926868,"replyedComment"],[3926869,"replyedComment"],[3926870,"replyedComment"],[3926871,"replyedComment"],[3926872,"replyedComment"],[3926873,"replyedComment"],[3926874,"replyedComment"],[3926875,"replyedComment"],[3926876,"replyedComment"],[3926877,"replyedComment"],[3926878,"replyedComment"],[3926879,"replyedComment"]] :put node_label {id_n,ln}')
res